# One-hot encoding in PySpark

PySpark에서 원-핫 인코딩을 수행하려면 다음을 수행해야 합니다.
1. StringIndexer를 사용하여 범주 열을 숫자 열(0, 1, ...)로 변환
2. OneHotEncoder를 사용하여 숫자 열을 원-핫 인코딩 열로 변환

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataFrame").config("spark.sql.repl.eagerEval.enabled", True).getOrCreate()

In [2]:
# 데이터를 데이터프레임으로 변환합니다.
rows = [['Alex', 'B'], ['Bob', 'A'], ['Cathy', 'B'], ['Dave', 'C'], ['Eric', 'D']]

df = spark.createDataFrame(rows, ['name', 'class'])
df

name,class
Alex,B
Bob,A
Cathy,B
Dave,C
Eric,D


## class 를 One Hot Encoding
### 1 단계 - StringIndexer를 사용하여 클래스 열을 숫자 열로 변환

- PySpark의 StringIndexer는 문자열로 된 범주형 데이터를 숫자형으로 변환할 때, **각 범주의 빈도수(Frequency)**를 기준으로 인덱스를 부여. 가장 빈도가 높은 값에 0을 부여하고, 그다음 빈도에 1, 2 등의 순서로 인덱스를 부여

In [3]:
# 문자열 데이터를 숫자로 인덱싱
from pyspark.ml.feature import StringIndexer

# StringIndexer는 문자열로 이루어진 컬럼을 숫자형 인덱스로 변환
# inputCol: 인덱싱할 입력 컬럼
# outputCol: 변환된 인덱스를 저장할 출력 컬럼
# 데이터프레임(df)을 기반으로 StringIndexer 모델을 학습시킵니다.
indexer = StringIndexer(inputCol='class', outputCol='class_').fit(df)

# 학습된 StringIndexer 모델을 사용하여 원본 데이터프레임(df)에 변환을 적용
df_r = indexer.transform(df)
df_r

name,class,class_
Alex,B,0.0
Bob,A,1.0
Cathy,B,0.0
Dave,C,2.0
Eric,D,3.0


In [4]:
indexer.labels

['B', 'A', 'C', 'D']

In [5]:
df_r.groupBy('class').count().show()

+-----+-----+
|class|count|
+-----+-----+
|    B|    2|
|    A|    1|
|    D|    1|
|    C|    1|
+-----+-----+



### 2 단계 : One-Hot Encoding

희소 벡터는 다음 세 가지 값으로 정의됩니다.

    - size: 벡터의 크기(카테고리 수에서 1을 뺀 값)  
    - 인덱스: 값을 보유하는 벡터의 인덱스  
    - 값: 인덱스에 있는 값  

4개의 고유한 범주(A,B,C,D) 를 예를 들면 하나의 범주가 기본 범주로 사용되기 때문에 벡터의 크기는 3입니다.  
```
A → [1, 0, 0] → (3, [0], [1.0])
B → [0, 1, 0] → (3, [1], [1.0])
C → [0, 0, 1] → (3, [2], [1.0])
D → [0, 0, 0] → (3, [], [])
```
벡터 (3,[0],[1.0])를 예로 들면, 두번째 값 [0]과 세 번째 값 [1.0]은 벡터에서 인덱스 위치 0을 1.0으로 채워야 함을 의미합니다. 희소 벡터의 다른 모든 값은 0으로 채워집니다.  

원-핫 인코딩된 벡터(3,[],[])의 두 번째 및 세 번째 값은 모두 비어 있습니다. 이것은 벡터가 0으로 채워져 있음을 의미합니다. 즉, 범주 D가 기본 범주로 취급됩니다. 이것이 크기 3의 벡터로 4개의 고유 범주를 나타낼 수 있는 비결입니다.

In [6]:
# OneHotEncoder는 숫자형 인덱스를 희소 벡터(Sparse Vector)로 변환
from pyspark.ml.feature import OneHotEncoder

# inputCols: 원-핫 인코딩을 적용할 입력 컬럼
# outputCols: 변환된 원-핫 벡터를 저장할 출력 컬럼
# fit: 데이터프레임(df_r)을 기반으로 원-핫 인코딩 모델을 학습
encoder = OneHotEncoder(inputCols=['class_'], outputCols=['class_ohe']).fit(df_r)

# 학습된 원-핫 인코딩 모델을 사용하여 데이터를 변환합니다.
df_onehot = encoder.transform(df_r)
df_onehot

name,class,class_,class_ohe
Alex,B,0.0,"(3,[0],[1.0])"
Bob,A,1.0,"(3,[1],[1.0])"
Cathy,B,0.0,"(3,[0],[1.0])"
Dave,C,2.0,"(3,[2],[1.0])"
Eric,D,3.0,"(3,[],[])"


In [7]:
# 데이터프레임(df_onehot)에서 상위 5개의 행을 리스트로 반환
df_onehot.head(5)

[Row(name='Alex', class='B', class_=0.0, class_ohe=SparseVector(3, {0: 1.0})),
 Row(name='Bob', class='A', class_=1.0, class_ohe=SparseVector(3, {1: 1.0})),
 Row(name='Cathy', class='B', class_=0.0, class_ohe=SparseVector(3, {0: 1.0})),
 Row(name='Dave', class='C', class_=2.0, class_ohe=SparseVector(3, {2: 1.0})),
 Row(name='Eric', class='D', class_=3.0, class_ohe=SparseVector(3, {}))]

In [8]:
# 데이터프레임(df_onehot)의 스키마(schema) 출력
df_onehot.printSchema()

root
 |-- name: string (nullable = true)
 |-- class: string (nullable = true)
 |-- class_: double (nullable = false)
 |-- class_ohe: vector (nullable = true)



### VectorAssembler는 머신러닝 모델에서 사용할 특징(feature) 벡터 생성
- 여러 개의 입력 컬럼을 하나의 벡터로 결합

In [9]:
from pyspark.ml.feature import VectorAssembler

# inputCols: 벡터로 결합할 입력 컬럼들의 리스트
# outputCol: 결합된 벡터를 저장할 출력 컬럼
assembler = VectorAssembler(inputCols=['class_ohe'], outputCol='features')

# 학습된 VectorAssembler를 사용하여 데이터프레임(df_onehot)에 변환을 적용
output = assembler.transform(df_onehot)
output

name,class,class_,class_ohe,features
Alex,B,0.0,"(3,[0],[1.0])","[1.0,0.0,0.0]"
Bob,A,1.0,"(3,[1],[1.0])","[0.0,1.0,0.0]"
Cathy,B,0.0,"(3,[0],[1.0])","[1.0,0.0,0.0]"
Dave,C,2.0,"(3,[2],[1.0])","[0.0,0.0,1.0]"
Eric,D,3.0,"(3,[],[])","(3,[],[])"


In [11]:
output.select("features")

features
"[1.0,0.0,0.0]"
"[0.0,1.0,0.0]"
"[1.0,0.0,0.0]"
"[0.0,0.0,1.0]"
"(3,[],[])"
